# I. CHOMAGE

# Imstallation et importation des packages 

In [ ]:
""" %pip install numpy
%pip install pandas
%pip install matplotlib
%pip install openpyxl
%pip install pycodestyle
%pip install pycountry
%pip install seaborn
%pip install pickleshare"""
import pycodestyle as pep8
import openpyxl as xl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pycountry
import seaborn as sns
np.random.seed(123)

# Importation de la BD

In [ ]:
rate=pd.read_excel("C:/Users/yousr/Downloads/Projet_py/Unemployment Rate.xlsx", sheet_name='monthly' , index_col=0 )
rate.head(5)
rate.index = pd.to_datetime(rate.index,format='%YM%m')
rate.index =rate.index.strftime('%Y-%m')

In [ ]:
# Noms des colonnes
col_names=rate.columns
print(col_names)

# Type de données

In [ ]:
# Type de données de chaque colonne 
datas_type= rate.dtypes
print(datas_type)

# Doublons

In [ ]:
duplicates = rate[rate.duplicated()]
duplicates.head(5) # 0 doublons dans notre DF

# Erreurs de frappes pour la colonne index

In [ ]:
rate.index = pd.to_datetime(rate.index, errors='coerce')
errors = rate[rate.index.isnull()]
errors.head(10) # Pas d'erreurs #########################################################################################

# Construction de DF

In [ ]:
# Garder uniquement les pays dans la DF : Méthode search_fuzzy de pycountry ###############################################################

def detect_countries(col_names):
    countries_detected = []
    
    for name in col_names:
        try:
            # Essayer de trouver le pays par son nom
            pays= pycountry.countries.search_fuzzy(name)[0]
            countries_detected.append(pays.name)
        except LookupError:
            # Ignorer les noms qui ne correspondent à aucun pays
            pass
    
    return countries_detected

# Liste des pays detectés

countries_detected = detect_countries(col_names)
print("Noms de pays détectés :", countries_detected)

# Longeur des listes : countries_detected et col_names
print(len(countries_detected))
print(len(col_names))

# Différence entre les deux listes pour obtenir les noms des colonnes ignorées par la fonction detect_countries
#L'objectif est de vérifier si des pays ont été omis par la fonction detect_countries

# Pays dans col_names et pas dans countries_detected
diff= list(set(col_names) - set(countries_detected))
print(diff) 

# Pays dans countries_detected et pas dans col_names
diff2= list(set(countries_detected)- set(col_names))
print(diff2)

countries_omitted =['Korea, Rep.', 'Taiwan, China', 'Hong Kong SAR, China', 'Czech Republic', 'Egypt, Arab Rep.', 'Venezuela, RB'] ####################""
concat = countries_detected + countries_omitted 

# Enfin, la liste finale des pays à retenir dans la DF
countries_detected =list(set(concat)- set(diff2))
print(countries_detected)

# La DF taux de chomage à retenir
rate_filter= rate[countries_detected]
rate_filter.columns

In [ ]:
# Pour faciliter la lecture de la DF, on remplace les noms des pays par leurs codes ISO  correspondant

# Dictionnaire de correspondance entre noms complets des pays et leurs codes
corresp = {country.name: country.alpha_3 for country in pycountry.countries}

# Liste initiale des noms complets des colonnes
country = countries_detected
# Transformation des noms complets des colonnes en abréviations
country_codes = [corresp.get(pays, pays) for pays in country]

# Remplacement des noms des pays par leurs codes dans la merged
rate_filter.columns=country_codes
rate_filter.columns
rate_filter.head(5)

#Ordre alphabétique des colonnes 
rate_filter_sort= rate_filter.sort_index(axis=1)
rate_filter_sort.head(5)

# Visualisation des NA

In [ ]:
# Visualisation des valeurs manquantes NA
import missingno as msno
# Diagramme à barres des valeurs manquantes
msno.bar(rate_filter_sort)
plt.show()

# % des NA

In [ ]:
# Calculer le taux de valeurs manquantes dans chaque colonne
size = rate_filter_sort.shape
nan_sum = rate_filter_sort.isna().sum() # Total du nombre de NA par colonne
nan_percent = nan_sum.sort_values(ascending=True) * 100 / size[0]
print(nan_percent)

# Seuil des NA

In [ ]:
# Récupérer les colonnes avec un pourcentage de valeurs manquantes supérieur à 40%
cols_to_drop = nan_percent[nan_percent > 40].index
print(cols_to_drop)

# Supprimer les colonnes de la DataFrame
rate_filter_sort.drop(cols_to_drop, axis=1, inplace=True)

In [ ]:
# Type de données de chaque colonne 
data_type = rate_filter_sort.dtypes
print(data_type)

# Imputation

In [ ]:
# Imputation des NA

#rate_filter_sort_inter=rate_filter_sort.interpolate(method='linear', axis=0, limit=None, inplace=False, limit_direction=None, limit_area=None)

#rate_filter_sort.fillna(value=None, method='ffill', axis=None, inplace=False, limit=None)

rate_no_na = rate_filter_sort.fillna(rate_filter_sort.mean())

#rate_filter_sort_fill= rate_filter_sort.iloc[-5:].ffill()

indice_premiere_na = rate_filter_sort.isna().any(axis=1).idxmax()
print(indice_premiere_na)

#numero_ligne = rate_filter_sort.index.get_loc(indice_premiere_na) + 1
"""
from datetime import datetime
date_today = datetime.now()

# Extrait l'année de la date actuelle
current_year = date_today.year
str(current_year)
rate_filter_sort.index = pd.to_datetime(rate_filter_sort.index, format='mixed')


rate_filter_sort.index = rate_filter_sort.index.astype(str)
rate_current_year = rate_filter_sort[rate_filter_sort.index.str.contains(current_year)]


#rate_filter_sort.index = pd.to_datetime(rate_filter_sort.index)

#year2023 =rate_filter_sort.loc[rate_filter_sort.index.year == 2023]

#.fillna(rate_filter_sort.mean())
print(rate_filter_sort.index.dtype) """

# Valeurs abérrantes

In [ ]:
# Les valeurs abérrantes 
plt.figure(figsize=(10, 6))
sns.boxplot(data=rate_no_na)
plt.title('Boîtes à moustaches des séries temporelles')
plt.show()

# Ajustement

In [ ]:
rate_no_na.index = pd.to_datetime(rate_no_na.index, format='%Y-%m-%d')

# Grouper par année et vérifier si tous les mois sont présents
rate_12= rate_no_na.groupby(rate_no_na.index.year).filter(lambda x: len(x) == 12)
rate_12= pd.DataFrame(rate_12)
rate_12.head(5)

# Regroupement

In [ ]:
rate_quart = rate_12.resample('Q-JAN').mean()
#print(rate_quart)

# Jointure

In [ ]:
# Ignorer les jours dans l'index
rate_quart.index =rate_quart.index.strftime('%Y-%m')

# DF classification

# Pivotage

In [ ]:
#result_table = merged.melt(id_vars='merged.index', var_name='Country', value_name='Value')
# result_table = merged.pivot(index='gdp_no_na.index', columns='rate_quart', values='gdp_no_na')


# II. PIB

In [ ]:
#%pip install missingno
import pycodestyle as pep8
import openpyxl as xl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pycountry
import missingno as msno
import seaborn as sns
np.random.seed(123)

# Importation 

In [ ]:
# Importation du fichier .xlsx
gdp=pd.read_excel("C:/Users/yousr/Downloads/Projet_py/GDP Deflator at Market Prices, LCU.xlsx",  sheet_name='quarterly', index_col=0)
gdp.head(5)
gdp.index = pd.to_datetime(gdp.index)
gdp.index =gdp.index.strftime('%Y-%m')

In [ ]:
# Noms des colonnes
col_names=gdp.columns
print(col_names)

# Type de données

In [ ]:
# Type de données de chaque colonne 
datas_type= gdp.dtypes
print(datas_type)

# Doublons

In [ ]:
duplicates = gdp[gdp.duplicated()]
duplicates.head(5) # 0 doublons dans notre gdp_no_na

# Erreurs de frappes pour la colonne index

In [ ]:
gdp.index = pd.to_datetime(gdp.index, errors='coerce')
errors = gdp[gdp.index.isnull()]
print(errors) # Pas d'erreurs

# Type de données

# Type de données de chaque colonne 
datas_type= gdp.dtypes
print(datas_type)

# Construction de la DF

In [ ]:
# Garder uniquement les pays dans la gdp_filter_sort : Méthode search_fuzzy de pycountry ###############################################################

def detect_countries(col_names):
    countries_detected = []
    
    for name in col_names:
        try:
            # Essayer de trouver le pays par son nom
            pays= pycountry.countries.search_fuzzy(name)[0]
            countries_detected.append(pays.name)
        except LookupError:
            # Ignorer les noms qui ne correspondent à aucun pays
            pass
    
    return countries_detected

# Liste des pays detectés

countries_detected = detect_countries(col_names)
print("Noms de pays détectés :", countries_detected)

# Longeur des listes : countries_detected et col_names
print(len(countries_detected))
print(len(col_names))

# Différence entre les deux listes pour obtenir les noms des colonnes ignorées par la fonction detect_countries
#L'objectif est de vérifier si des pays ont été omis par la fonction detect_countries

# Pays dans col_names et pas dans countries_detected
diff= list(set(col_names) - set(countries_detected))
print(diff) 

# Pays dans countries_detected et pas dans col_names
diff2= list(set(countries_detected)- set(col_names))
print(diff2)

countries_omitted =['Korea, Rep.', 'Czech Republic', 'Taiwan, China', 'Bolivia', 'Hong Kong SAR, China', 'Egypt, Arab Rep.'] ####################""
concat = countries_detected + countries_omitted 

# Enfin, la liste finale des pays à retenir dans la gdp_filter_sort
countries_detected =list(set(concat)- set(diff2))
print(countries_detected)

# La gdp_filter_sort taux de chomage à retenir
gdp_filter= gdp[countries_detected]
gdp_filter.columns


In [ ]:
# Pour faciliter la lecture de la gdp_filter_sort, on remplace les noms des pays par leurs codes ISO  correspondant

# Dictionnaire de correspondance entre noms complets des pays et leurs codes
corresp = {country.name: country.alpha_3 for country in pycountry.countries}

# Liste initiale des noms complets des colonnes
country = countries_detected
# Transformation des noms complets des colonnes en abréviations
country_codes = [corresp.get(pays, pays) for pays in country]

# Remplacement des noms des pays par leurs codes dans la gdp_filter_sort
gdp_filter.columns=country_codes
gdp_filter.columns
gdp_filter.head(5)

#Ordre alphabétique des colonnes 
gdp_filter_sort= gdp_filter.sort_index(axis=1)
gdp_filter_sort.head(5)

In [ ]:
# Nombre de lignes d'une gdp_filter_sort
count_rows= len(gdp_filter_sort)
print(count_rows)

# Visualisation

In [ ]:
# Diagramme à barres des valeurs manquantes
msno.bar(gdp_filter_sort)
plt.show()

# % des NA

In [ ]:
# Calculer le taux de valeurs manquantes
size = gdp_filter_sort.shape
nan_values = gdp_filter_sort.isna().sum()
nan_percent = nan_values.sort_values(ascending=True) * 100 / size[0]
print(nan_percent)


# Seuil des NA 

In [ ]:
# Récupérer les colonnes avec un pourcentage de valeurs manquantes supérieur à 40%
cols_to_drop = nan_percent[nan_percent > 40].index
print(cols_to_drop)

# Supprimer les colonnes de la DataFrame
gdp_filter_sort.drop(cols_to_drop, axis=1, inplace=True)


In [ ]:
# Convertir l'index en type date { Les données sont trimestriels}
gdp_filter_sort.index = pd.to_datetime(gdp_filter_sort.index)

# Afficher le DataFrame
print(gdp_filter_sort)

In [ ]:
plt.figure(figsize=(15, 8))

for col in gdp_filter_sort.columns:
    plt.plot(gdp_filter_sort.index, gdp_filter_sort[col], label=col, alpha=0.7)

# Ajouter des titres et une légende
plt.title('Séries temporelles pour chaque colonne de la DataFrame')
plt.xlabel('Date')
plt.ylabel('Valeur')
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))

# Faire pivoter les étiquettes de l'axe des x
plt.xticks(rotation=45, ha='right')

# Afficher le graphique
plt.tight_layout()
plt.show()


# Imputation 

In [ ]:
# Imputation des valeurs manquantes NA par interpolation

# Interpolation linéaire :

# gdp_filter_sort.interpolate(method='linear', limit=None)

# Interpolation polynômiale d'ordre 3 :

# gdp_filter_sort.interpolate(method='polynomial', order=3, limit=None)
              
gdp_no_na = gdp_filter_sort.fillna(gdp_filter_sort.mean())

# Valeurs abérrantes 

In [ ]:
# Les valeurs abérrantes 
"""plt.figure(figsize=(10, 6))
sns.boxplot(data=gdp_no_na)
plt.title('Boîtes à moustaches des séries temporelles')
plt.show()"""

In [ ]:
gdp_no_na.index = pd.to_datetime(gdp_no_na.index, format='%Y-%m')

# Grouper par année et vérifier si tous les trimestres sont présents
gdp_no_na = gdp_no_na.groupby(gdp_no_na.index.year).filter(lambda x: len(x) == 4)
gdp_no_na = pd.DataFrame(gdp_no_na)
gdp_no_na.head(5)

In [ ]:
%pip install tensorflow==2.7.0 
%pip install scikit-learn

In [ ]:
# Fusionner les DataFrames sur l'index en utilisant une jointure interne (how='inner')
common_columns = rate_quart.columns.intersection(gdp_no_na.columns) # Pour garder uniquement les colonnes présentes dans les deux DF
merged= pd.merge(rate_quart[common_columns], gdp_no_na[common_columns], left_index=True, right_index=True, how='inner' )
merged.head(5)

In [ ]:
print(rate_quart.index)
print(gdp_no_na.index)
print(rate_quart.index.equals(gdp_no_na.index))

In [ ]:
mean_rate = pd.DataFrame(rate_quart.mean(), columns=['Taux de chomage'])
mean_gdp = pd.DataFrame(gdp_no_na.mean(), columns=['PIB'])
merged_mean= pd.merge(mean_rate, mean_gdp, left_index=True, right_index=True, how='inner')

# III. Education

# Importation 

In [ ]:
educ=pd.read_excel("C:/Users/yousr/Downloads/Projet_py/BIg_data.xlsx", sheet_name='HNP_StatsData', header=1, index_col=0)
educ.head(5)
col_names=educ.columns
print(col_names)

# Life Expectancy

In [ ]:
masque = educ['Indicator Name'].str.contains('expectancy', case=False) & educ['Indicator Name'].str.contains('total', case=False)
# Appliquez le masque pour obtenir un nouveau DataFrame avec les lignes filtrées
educ_LE = educ[masque]

In [ ]:
# Filtrer les colonnes qui contiennent des années ou les noms des pays
cols_to_keep = educ_LE.columns[educ_LE.columns.str.contains(r'\d{4}|Indicator Name')]

# Créer un nouveau DataFrame avec les colonnes filtrées
educ_LE_col = educ_LE[cols_to_keep]
print(educ_LE_col)

In [ ]:
educ_LE_col = educ_LE_col.drop(columns="Indicator Name")
mean_LE = educ_LE_col.mean(axis=1)

In [ ]:
mean_LE =mean_LE.to_frame(name='Life Expectancy')

# Population growth rate

In [ ]:
masque1 = educ['Indicator Name'].str.contains('Population growth ', case=False)
educ_PGR = educ[masque1]

In [ ]:
# Filtrer les colonnes qui contiennent des années ou les noms des pays
cols_to_keep1 = educ_PGR.columns[educ_PGR.columns.str.contains(r'\d{4}|Indicator Name')]

# Créer un nouveau DataFrame avec les colonnes filtrées
educ_PGR_filter1 = educ_PGR[cols_to_keep1]
print(educ_PGR_filter1)

In [ ]:
unique = educ_PGR_filter1['Indicator Name'].unique()
print(unique)

# On souhaite garder que le taux de croissance de la population totale

# Par élimination successive
# On garde les lignes où il y a le taux de croissance du milieu hors que urbain
masque2 = educ_PGR_filter1['Indicator Name'].str.contains('urban', case=False)
masque2_inverse= ~masque2 # Récupérer les lignes qui ne vérifient pas le masque
educ_PGR_filter2 = educ_PGR_filter1[masque2_inverse]

# On garde les lignes où il y a le taux de croissance du milieu hors que rural
masque3 = educ_PGR_filter2['Indicator Name'].str.contains('rural', case=False)
masque3_inverse = ~masque3
educ_PGR_filter3 = educ_PGR_filter2[masque3_inverse]

In [ ]:
educ_PGR_filter3  = educ_PGR_filter3.drop(columns="Indicator Name")
mean_PGR = educ_PGR_filter3.mean(axis=1)

In [ ]:
mean_PGR=mean_PGR .to_frame(name='Life Expectancy')

# IV. GMD Inégalités

# Importation des packages

In [ ]:
import pycodestyle as pep8
import openpyxl as xl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pycountry
np.random.seed(123)

# Importation de la BD

In [ ]:
# Gini Mean Difference
gmd=pd.read_excel("C:/Users/yousr/Downloads/Projet_py/inequality GMD World Bank.xlsx",  sheet_name='data', index_col=0)
gmd.head(5)
col_names=gmd.columns
print(col_names)

# Extraction des colonnes nécessaires

In [ ]:
# Base de données de l'indice Theil
theil = gmd.pivot(index='year', columns='countryname', values='index')
theil.head(5)

In [ ]:
# La moyenne de l'indice de theil sur toutes les périodes
mean_theil = theil.mean(axis=0) 

In [ ]:
# Base de données du taux d'urbanisation
urban = gmd.pivot(index='year', columns='countryname', values='sp_urb_totl_in_zs')
urban.head(5)

In [ ]:
# La moyenne du taux d'urbanisation sur toutes les périodes
mean_urban = urban.mean(axis=0)

In [ ]:
# Base de données du ratio de pauvreté
poverty = gmd.pivot(index='year', columns='countryname', values='si_pov_lmic')
poverty.head(5)

In [ ]:
# La moyenne du ratio de pauvreté sur toutes les périodes
mean_poverty = poverty.mean(axis=0)

In [ ]:
# Dans pandas, une Df à une colonne correspond à une série. On donne un nom à chaque série pour une éventuelle jointure
mean_poverty.name = 'Ratio de pauvreté'
mean_urban.name = "Taux urbanisation"
mean_theil.name = 'Theil'

# Jointure 

In [ ]:
# Jointure entre df1 et df2 sur la colonne "countryname"
merged1 = pd.merge(mean_poverty, mean_urban, on='countryname', how='inner')
# Jointure entre le résultat précédent (merged_df) et df3 sur la colonne "countryname"
final_merged1 = pd.merge(merged1, mean_theil, on='countryname', how='inner')

In [ ]:
# Dans pandas, une Df à une colonne correspond à une série. On donne un nom à chaque série pour une éventuelle jointure
mean_LE.name = 'Espérance de vie'
mean_PGR.name = "Croissance démo"

In [ ]:
final_merged1.index.name = 'Country Name'  ####################################################################################
mean_LE.index.name = 'Country Name'
mean_PGR.index.name = 'Country Name'
merged_mean.index.name = 'Country Name'

In [ ]:
# Jointure entre df1 et df2 sur la colonne "countryname"
merged2 = pd.merge(final_merged1, mean_PGR, on='Country Name', how='inner')
# Jointure entre le résultat précédent (merged_df) et df3 sur la colonne "countryname"
merged3 = pd.merge(merged2, mean_LE, on='Country Name', how='inner')

# Transposé de merged3

In [ ]:
merged3_transpose = merged3.transpose()
col_names = merged3_transpose.columns
merged3_transpose.head(5)

# Code ISO des pays

In [ ]:
# Pour faciliter la lecture de la DF, on remplace les noms des pays par leurs codes ISO  correspondant

# Dictionnaire de correspondance entre noms complets des pays et leurs codes
corresp = {country.name: country.alpha_3 for country in pycountry.countries}

# Liste initiale des noms complets des colonnes
country = col_names
# Transformation des noms complets des colonnes en abréviations
country_codes = [corresp.get(pays, pays) for pays in country]

# Remplacement des noms des pays par leurs codes dans la merged
merged3_transpose.columns=country_codes
merged3_transpose.columns
merged3_transpose.head(5)

#Ordre alphabétique des colonnes 
merged3_transpose_sort= merged3_transpose.sort_index(axis=1)
merged3_transpose_sort.head(5)

In [ ]:
merged4=merged3_transpose.transpose()
merged4.index.name= 'Country Name'
merged4.head(5)

In [ ]:
final_merged= pd.merge(merged4, merged_mean, on='Country Name', how='inner')